In [80]:
import httpx
import json
import lxml.html
import time
from urllib.parse import urlparse
import ast
import re


In [83]:
headers = {
    "authority": "www.zillow.com",
    "method": "GET",
    "scheme": "https",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "max-age=0",
    "cookie": "zguid=24|%24a54d78f6-9268-4405-9be6-3f3663abd80e",  # Replace with your actual cookies if needed
    "priority": "u=0, i",
    "sec-ch-ua": '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"
}

def scrape_property(url: str):
    with httpx.Client(headers = headers, follow_redirects=True) as client:
        response = client.get(url)
        property = lxml.html.fromstring(response.text)
        property_content = property.get_element_by_id("__NEXT_DATA__").text
        json_data = json.loads(property_content)
        # zpid, latitude, longitude, regionid, regiontype:
        # print(json_data["props"]["pageProps"]["componentProps"]["searchPageStateParams"])
        print(json_data["props"]["pageProps"]["componentProps"]["gdpClientCache"])
    return json_data["props"]["pageProps"]["componentProps"]["gdpClientCache"]

In [84]:
# Target URL
target_url = "https://www.zillow.com/homedetails/8430-S-Maryland-Ave-Chicago-IL-60619/4021370_zpid/"
property = scrape_property(target_url)
type(property)

{"ForRentShopperPlatformFullRenderQuery{\"zpid\":4021370,\"altId\":null}":{"property":{"zpid":4021370,"city":"Chicago","state":"IL","homeStatus":"FOR_RENT","address":{"streetAddress":"8430 S Maryland Ave","city":"Chicago","state":"IL","zipcode":"60619","neighborhood":null,"community":null,"subdivision":null},"isListingClaimedByCurrentSignedInUser":false,"isCurrentSignedInAgentResponsible":false,"bedrooms":4,"bathrooms":2,"price":2400,"yearBuilt":null,"streetAddress":"8430 S Maryland Ave","zipcode":"60619","listingAccount":null,"ssid":100066,"isCurrentSignedInUserVerifiedOwner":false,"editPropertyHistorylink":null,"propertyUpdatePageLink":null,"moveHomeMapLocationLink":null,"propertyEventLogLink":null,"responsivePhotos":[{"caption":"","subjectType":null,"url":"https://photos.zillowstatic.com/fp/7eca6235b0bddd047b377df31d257b3e-p_d.jpg","mixedSources":{"jpeg":[{"url":"https://photos.zillowstatic.com/fp/7eca6235b0bddd047b377df31d257b3e-cc_ft_192.jpg","width":192},{"url":"https://photos.zi

str

In [79]:
def string_to_dict(s):
    return ast.literal_eval(s)

s = "property"
print(string_to_dict(s))

ValueError: malformed node or string: <ast.Name object at 0x11ce30670>

In [74]:
def extract_keys(obj, path="", keys_paths=None):
    """Recursively extracts keys and their paths from a JSON object."""
    if keys_paths is None:
        keys_paths = {}
    
    if isinstance(obj, dict):
        for key, value in obj.items():
            new_path = f"{path}.{key}" if path else key
            keys_paths[key] = new_path
            extract_keys(value, new_path, keys_paths)
    elif isinstance(obj, list):
        for index, item in enumerate(obj):
            new_path = f"{path}[{index}]"
            extract_keys(item, new_path, keys_paths)
    
    return keys_paths

def print_json_keys(json_data):
    """Takes a JSON dictionary and prints all keys with their paths."""
    if not isinstance(json_data, dict):
        raise ValueError("Input must be a dictionary.")
    keys_paths = extract_keys(json_data)
    for path in keys_paths.values():
        print(path)

def find_key_path(json_data, target_key):
    """Finds and returns the path for a specific key in the JSON dictionary."""
    if not isinstance(json_data, dict):
        raise ValueError("Input must be a dictionary.")
    keys_paths = extract_keys(json_data)
    return keys_paths.get(target_key, None)
